In [1]:
## Import necessary packages
import pandas as pd
import numpy as np
import tkinter as tk
import PyPDF2 
import tabula
import tkinter
import camelot
import ghostscript
import re
import os
import csv
import urllib
import io
import warnings
warnings.filterwarnings("ignore")

## Minnesota
NOTE: This scrapes the PPL when in fact the desired document is the table within the final IUP.
This final IUP was provided by state administrators as an excel spreadsheet and thus this process is no longer necessary,
but the code is kept as a potential resource for future use.

In [188]:
## (23) MINNESOTA
# get the full table
mn_pdfs = camelot.read_pdf("https://www.health.state.mn.us/communities/environment/water/docs/dwrf/priority2023.pdf", pages='all', flavor='stream')

# create list to append into
list_of_dfs = []

# fix issue where sometimes the first two columns come in merged together 
# and sometimes the '2' column is empty space that throws off merging pages together
# by merging together these columns as needed, then when they are uniform across all pages, process once

for i in range(len(mn_pdfs)):
    # read in each page
    page_df = mn_pdfs[i].df.iloc[2:,].reset_index(drop = True)
    if len(page_df.columns) == 7:
        # combine 0 and 1 (to split later, but it's easier than trying to find some that are seperate and others that arent)
        page_df['rank_name'] = page_df[[0, 1]].agg(' '.join, axis=1)
        page_df.drop(columns=[0,1], inplace=True)
        page_df.columns = ["Project", "Project.ID", "Points", "Population", "Project.Cost", "Rank.Name"]
    
    elif len(page_df.columns) == 8:
        # combine 0 and 1 and 2 (because 2 is empty, this is the same as 0+1 above)
        page_df['rank_name'] = page_df[[0, 1, 2]].agg(' '.join, axis=1)
        page_df.drop(columns=[0,1,2], inplace=True)
        page_df.columns = ["Project", "Project.ID", "Points", "Population", "Project.Cost", "Rank.Name"]

    else:
        # len=6, rank and name are already together, just rename
        page_df.columns = ["Rank.Name", "Project", "Project.ID", "Points", "Population", "Project.Cost"]

    # append to list    
    list_of_dfs.append(page_df)
    
# concat with common column names    
mn = pd.concat(list_of_dfs)

# remove line breaks from the rank_name combo column
mn['Rank.Name'] = mn['Rank.Name'].str.replace("\n", "")

#split rank and name into two columns, drop original column
# regex = match digits greedily, a white space, then all remaining characters
# (eg '543 Oak Grove' splits to ['543', 'Oak Grove'])
mn[['Rank', 'Water.System']] = mn['Rank.Name'].str.extract(r'(\d+)\s(.*)', expand=True)
mn.drop(columns=['Rank.Name'], inplace=True)

# drop remaining column header row
mn = mn.query("Points != 'PROJECT ID  POINTS  POPULATION'").copy()

# drop the grand total row at the end
mn = mn.iloc[:-1,].reset_index(drop=True)

#reorder columns
mn = mn[["Rank", "Water.System", "Project", "Project.ID", "Points", "Population", "Project.Cost"]].copy()
mn.sample(5)

,Rank,Water.System,Project,Project.ID,Points,Population,Project.Cost
0,1,Westbrook,Treatment - New RO Plant for Nitrite,1170005-2,30,900,"$6,890,100"
1,2,Clitherall,New System - NO3 Connect to Battle Lake,1560033-2,30,25,"$9,634,500"
2,3,Hayward,Treatment - Radium Treatment & New Well,1240012-1,30,227,"$3,528,000"
3,4,Danube,Treatment - New Ra/Fe/Mn Plant,1650004-2,30,493,"$4,787,000"
4,5,Rush City,Treatment - Plant Upgrade,1130013-7,30,"3,175","$287,500"
...,...,...,...,...,...,...,...
669,670,Carver,"Storage - Recoat 100,000 Gal Tower",1100017-3,5,"1,300","$272,000"
670,671,Minnetrista,Storage - New 0.5 MG Tower,1270036-4,5,"2,913","$2,400,000"
671,672,Minnetrista,Watermain - South/Central Connection,1270036-5,5,"2,913","$2,800,000"
672,673,Tonka Bay,Watermain - Replacement,1270013-2,5,"1,570","$3,035,795"
